The elo system for two players $A$ and $B$ with $E$ being the expected score (chance of winning in percent) and $R$ being the rating of the players says:
$$E_{A}={\frac {1}{1+10^{(R_{B}-R_{A})/400}}}$$
or conversely
$$E_{B}={\frac {1}{1+10^{(R_{A}-R_{B})/400}}}$$

Further, the adjustment after a match, with $R'$ being the updated rating, $S$ being the outcome, and $K$ being a scaling factor:
$$R_{A}^{\prime }=R_{A}+K(S_{A}-E_{A})$$
This scaling factor of $K$ represents the maximum the score can be adjusted by, and is apparently often adjusted depending on the player's current score. An example $K$ equation from the USCF, with $N_e$ being the effective number of games a rating is based on and $m$ being the number of games in a tournament:
$$K=800/(N_{e}+m)$$
Other implementations seem to be without formulas, e.g.:
$$K = \begin{cases}40 & \textrm{less than 30 games played as long as } R < 1200 \\ 20 & R \leq 1200 \\ 10 & R > 1200\end{cases}$$
For simplicity's sake, we will proceed with the latter cases-based way of finding K.

In addition to the above, we have added a simple points difference factor $P$, which uses the team scores $p_A$ and $p_B$:
$$P = \frac{\max(p_A, p_B) - \min(p_A, p_B)}{(p_A - p_B) / 2}$$
This extends the $R'$ calculation:
$$R_{A}^{\prime }=R_{A}+(1 + P) \times K(S_{A}-E_{A})$$

In our case, we track 2v2 escort and 3v3/4v4 manhunt games, so we have to calculate the team rankings. We use a weighted mean $m_w$ of the team's rankings in accordance to each player's difference $d$ from the opposing team's mean ranking $m_e$, whereby weights are calculated according to:
$$w = \frac{\mathrm{abs}(p - m_e)}{d}$$

As of season 2, we have added 10 so called "placement games", which replace the above formulas. During these 10 games, any win raises the player's rating by 75, while every loss lowers it by 25. To compensate, every player starts off with a ranking of 750.

Side note: We are aware of the Glicko-2 algorithm, however this would be overly complicated to implement due to platform restrictions for certain players creating scenarios where ranking adjustment is needlessly inflated.

In [1]:
def E(R):
    return ((1 + 10 ** ((R[1] - R[0]) / 400)) ** -1)

def R_1v1(R, S, K=None, N=1):
    if K is None:
        K = Kc(N, R[0])
    return R[0] + K * (S - E(R))

def new_R(R, S, E, t1, t2, K=None, N=None):
    if N > 10:
        if K is None:
            K = Kc(N, R)
        return R + K * (S - E) * (1 + score(t1, t2))
    else:
        if S == 1:
            return R + 50
        elif S == 0:
            return R - 10
        elif S == .5:
            return R
        else:
            raise ValueError("Broken outcome.")

def Kc(N, R):
    hi = 1200
    if N < 30 and R < hi:
        return 40
    elif R <= hi:
        return 20
    else: # R > hi
        return 10
    
def score(t1, t2):
    if t1 > t2:
        return (t1 - t2) / ((t1 + t2) / 2)
    elif t1 < t2:
        return (t2 - t1) / ((t1 + t2) / 2)
    else:
        return 0
    
def w_mean(rankings, rankings_o):
    mean = sum(rankings_o) / len(rankings_o)
    diffs = []
    for r in rankings:
        diffs.append(abs(r - mean))
    weights = []
    for d in diffs:
        try:
            weights.append(d / sum(diffs))
        except ZeroDivisionError:
            weights.append(0)
    w_sum = sum(weights)
    if w_sum == 0:
        w_sum = len(rankings)
        weights = [1] * len(rankings)
    return sum([rankings[_] * weights[_] for _ in range(len(rankings))]) / sum(weights), weights

An example wrapper:

In [2]:
ratings_db = {"Dellpit": [750, 0], "MrEirox": [750, 0], "Tha Fazz": [750, 0], "Jelko": [750, 0], "Lime": [750, 0], "DurZa": [750, 0]}
db_copy = {"Dellpit": [750, 0], "MrEirox": [750, 0], "Tha Fazz": [750, 0], "Jelko": [750, 0], "Lime": [750, 0], "DurZa": [750, 0]}

In [3]:
def team_ratings(team_1, team_2, outcome, score_1, score_2):
    
    # team sizes
    l = len(team_1)
    
    # make sure team_1 and team_2 have same length
    if l != len(team_2):
        raise ValueError("team_1 and team_2 must have the same length!")
    
    # set S value according to outcome
    if outcome == 1:
        S = [1, 0]
    elif outcome == 2:
        S = [0, 1]
    elif outcome == 0:
        S = [.5, .5]
    else:
        raise ValueError("outcome must be 1 for team_1, 2 for team_2, or 0 for a tie!")
    
    # read out player data from db
    t1 = []
    t2 = []
    for i in range(l):
        t1.append(ratings_db[team_1[i]])
        t2.append(ratings_db[team_2[i]])
    
    # calculate total rating for each team 
    R_old_1 = []
    R_old_2 = []
    
    for i in range(l):
        R_old_1.append((t1[i])[0])
        R_old_2.append((t2[i])[0])
    
    # calculate expected outcome for each team
    E_1 = E([w_mean(R_old_1, R_old_2)[0], w_mean(R_old_2, R_old_1)[0]])
    E_2 = 1 - E_1
    
    # update values in database
    for i in range(l):
        ratings_db[team_1[i]] = [((new_R(R=(t1[i])[0], S=S[0], E=E_1, N=(t1[i])[1], t1=score_1, t2=score_2))), (t1[i])[1] + 1]
        ratings_db[team_2[i]] = [((new_R(R=(t2[i])[0], S=S[1], E=E_2, N=(t2[i])[1], t1=score_1, t2=score_2))), (t2[i])[1] + 1]
        db_copy[team_1[i]] = [((new_R(R=(t1[i])[0], S=S[0], E=E_1, N=(t1[i])[1], t1=0, t2=0))), (t1[i])[1] + 1]
        db_copy[team_2[i]] = [((new_R(R=(t2[i])[0], S=S[1], E=E_2, N=(t2[i])[1], t1=0, t2=0))), (t2[i])[1] + 1]
        
#    print("Ratings successfully adjusted.")
    
    return

In [4]:
#for _ in range(1000):
#    team_ratings(team_1=["Dellpit"], team_2=["Tha Fazz"], outcome=1, score_1=30000, score_2=15000)
import random
history = []
for _ in range(15):
    players = ["Dellpit", "MrEirox", "Lime", "Tha Fazz", "Jelko", "DurZa"]
    t1 = random.sample(players, k=3)
    players.remove(t1[0])
    players.remove(t1[1])
    players.remove(t1[2])
    out = [random.randint(1, 2), random.randint(10000, 25000), random.randint(10000, 25000)]
    team_ratings(team_1=t1, team_2=players, outcome=out[0], score_1=out[1], score_2=out[2])
    history.append(out)

In [5]:
#team_ratings(team_1=["Dellpit"], team_2=["Tha Fazz"], outcome=1)
print("with scores: ", ratings_db)
print("without scores: ", db_copy)
print("outcomes: ", history)

with scores:  {'Dellpit': [830.2058098640643, 15], 'MrEirox': [1374.5978021162505, 15], 'Tha Fazz': [976.6087915349976, 15], 'Jelko': [1191.1470483945677, 15], 'Lime': [958.8529516054325, 15], 'DurZa': [819.7941901359357, 15]}
without scores:  {'Dellpit': [827.8553807322565, 15], 'MrEirox': [1374.0101948332983, 15], 'Tha Fazz': [978.9592206668054, 15], 'Jelko': [1188.7966192627598, 15], 'Lime': [961.2033807372403, 15], 'DurZa': [822.1446192677435, 15]}
outcomes:  [[2, 18560, 22450], [2, 10954, 11051], [2, 14539, 11550], [1, 12675, 21566], [1, 22387, 18136], [1, 13373, 21189], [2, 14926, 18842], [1, 24619, 11541], [2, 16130, 11977], [1, 17769, 13889], [2, 20061, 13743], [1, 11367, 19434], [1, 11560, 13849], [2, 16789, 13772], [1, 20547, 12622]]


In [6]:
example_matches = [{"mode": "escort", "team_1": ["Dellpit", "MrEirox"], "team_2": ["Tha Fazz", "Jelko"], "outcome": 1, "new": False, "scores": [10000, 15000]},
                   {"mode": "escort", "team_1": ["Dellpit", "MrEirox"], "team_2": ["Tha Fazz", "Jelko"], "outcome": 1, "new": True, "scores": [10000, 13000]},
                   {"mode": "escort", "team_1": ["Dellpit", "MrEirox"], "team_2": ["Tha Fazz", "Jelko"], "outcome": 0, "new": True, "scores": [10000, 19000]}, 
                   {"mode": "escort", "team_1": ["Dellpit", "MrEirox"], "team_2": ["Tha Fazz", "Jelko"], "outcome": 2, "new": True, "scores": [10000, 11000]}]

In [10]:
def new_matches(matches):
    for i in range(len(matches)):
        m = matches[i]
        if m["new"]:
            m["new"] = False
            team_ratings(team_1=m["team_1"], team_2=m["team_2"], outcome=m["outcome"], score_1=m["scores"][0], score_2=m["scores"][1])
    return

In [11]:
new_matches(example_matches)

In [325]:
example_matches

[{'mode': 'escort',
  'team_1': ['Dellpit', 'MrEirox'],
  'team_2': ['Tha Fazz', 'Jelko'],
  'outcome': 1,
  'new': False},
 {'mode': 'escort',
  'team_1': ['Dellpit', 'MrEirox'],
  'team_2': ['Tha Fazz', 'Jelko'],
  'outcome': 1,
  'new': False},
 {'mode': 'escort',
  'team_1': ['Dellpit', 'MrEirox'],
  'team_2': ['Tha Fazz', 'Jelko'],
  'outcome': 0,
  'new': False},
 {'mode': 'escort',
  'team_1': ['Dellpit', 'MrEirox'],
  'team_2': ['Tha Fazz', 'Jelko'],
  'outcome': 2,
  'new': False}]

In [12]:
ratings_db

{'Dellpit': [801.2416260942472, 17],
 'MrEirox': [1367.3567561737962, 17],
 'Tha Fazz': [1005.5729753048148, 17],
 'Jelko': [1220.1112321643848, 17],
 'Lime': [958.8529516054325, 15],
 'DurZa': [819.7941901359357, 15]}

In [5]:
import random
for _ in range(5):
    r = [1091, 971, 1075]
    ro = [700, 1150, 1250]
    print("rankings: ", r)
    print("opponents' rankings: ", ro)
    print("unweighted mean: ", sum(r) / len(r))
    print("opponents' unweighted mean: ", sum(ro) / len(ro))
    print("weighted mean: ", w_mean(r, ro)[0])
    print("weights: ", w_mean(r, ro)[1])
    print("\n")

rankings:  [1091, 971, 1075]
opponents' rankings:  [700, 1150, 1250]
unweighted mean:  1045.6666666666667
opponents' unweighted mean:  1033.3333333333333
weighted mean:  1040.6082474226805
weights:  [0.35670103092783534, 0.3855670103092777, 0.25773195876288696]


rankings:  [1091, 971, 1075]
opponents' rankings:  [700, 1150, 1250]
unweighted mean:  1045.6666666666667
opponents' unweighted mean:  1033.3333333333333
weighted mean:  1040.6082474226805
weights:  [0.35670103092783534, 0.3855670103092777, 0.25773195876288696]


rankings:  [1091, 971, 1075]
opponents' rankings:  [700, 1150, 1250]
unweighted mean:  1045.6666666666667
opponents' unweighted mean:  1033.3333333333333
weighted mean:  1040.6082474226805
weights:  [0.35670103092783534, 0.3855670103092777, 0.25773195876288696]


rankings:  [1091, 971, 1075]
opponents' rankings:  [700, 1150, 1250]
unweighted mean:  1045.6666666666667
opponents' unweighted mean:  1033.3333333333333
weighted mean:  1040.6082474226805
weights:  [0.356701